In [1]:
import os
import glob
import time

import numpy as np
import pandas as pd
import xarray as xr

import metpy.calc as mpcalc
from metpy.units import units

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.collections import LineCollection

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import math
import dask.dataframe as dd
from scipy.spatial import Delaunay

from uviz.utils.tools import find_TC_bbox
from uviz.datashader_tools.utils import datashader_wrapper
from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

from haversine import haversine_vector, Unit

# import holoviews as hv
# from holoviews import opts

# import geoviews as gv
# import geoviews.feature as gf

# import datashader as ds
# from holoviews.operation.datashader import rasterize as hds_rasterize
# from holoviews.operation.datashader import datashade as hds_datashade

# gv.extension("bokeh","matplotlib")

# opts.defaults(
#     opts.Image(width=1200, height=600),
#     opts.RGB(width=1200, height=600))

In [2]:
prototype_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
storm_1279_dir = os.path.join(prototype_dir, "VR28.NATL.EXT.CAM5.4CLM5.0.dtime900_storm_1279")
parent_28km = os.path.join(storm_1279_dir, '28km')
child_3km = os.path.join(storm_1279_dir, '3km')

h1c_files = glob.glob(os.path.join(child_3km, '*.h1.*.nc'))
h2c_files = glob.glob(os.path.join(child_3km, '*.h2.*.nc'))
h3c_files = glob.glob(os.path.join(child_3km, '*.h3.*.nc'))
h4c_files = glob.glob(os.path.join(child_3km, '*.h4.*.nc'))
h5c_files = os.listdir(os.path.join(child_3km, 'h5'))

# Dug in the attributes for the mesh, TODO: make function that finds it automatically
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [ ]:
start = time.time()
# Native grid files
h1cn_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' not in f])
h2cn_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' not in f])
h3cn_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' not in f])
h4cn_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' not in f])
#h5cn_ds = xr.open_mfdataset([f for f in h5c_files])
c_mesh_ds = xr.open_dataset(c_mesh, chunks={'nCells':10000, 'nVertices':100, 'nEdges':100})

# Regridded files
h1cr_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' in f])
h2cr_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' in f])
h3cr_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' in f])
h4cr_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' in f])
end = time.time()

print(f'Time elapsed: {end-start} seconds.')

In [22]:
start = time.time()
# Native grid files
h1cn_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' not in f], parallel=True)
h2cn_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' not in f], parallel=True)
h3cn_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' not in f], parallel=True)
h4cn_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' not in f], parallel=True)
#h5cn_ds = xr.open_mfdataset([f for f in h5c_files], parallel=True)
c_mesh_ds = xr.open_dataset(c_mesh, chunks={'nCells':1000, 'nVertices':100, 'nEdges':100})

# Regridded files
h1cr_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' in f], parallel=True)
h2cr_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' in f], parallel=True)
h3cr_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' in f], parallel=True)
h4cr_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' in f], parallel=True)
end = time.time()
print(f'Time elapsed: {end-start} seconds.')

Time elapsed: 4.460851430892944 seconds.


In [23]:
h2cn_ds

<xarray.Dataset>
Dimensions:         (time: 21, ncol: 835586, lev: 32, ilev: 33, nbnd: 2)
Coordinates:
  * ilev            (ilev) float64 4.5e+04 4.291e+04 4.085e+04 ... 249.0 0.0
  * lev             (lev) float64 4.395e+04 4.188e+04 3.984e+04 ... 476.3 124.5
  * time            (time) object 2008-08-30 00:00:00 ... 2008-09-04 00:00:00
Dimensions without coordinates: ncol, nbnd
Data variables: (12/59)
    CLDHGH          (time, ncol) float32 dask.array<chunksize=(4, 835586), meta=np.ndarray>
    CLDLOW          (time, ncol) float32 dask.array<chunksize=(4, 835586), meta=np.ndarray>
    CLDMED          (time, ncol) float32 dask.array<chunksize=(4, 835586), meta=np.ndarray>
    CLDTOT          (time, ncol) float32 dask.array<chunksize=(4, 835586), meta=np.ndarray>
    CLOUD           (time, lev, ncol) float32 dask.array<chunksize=(4, 32, 835586), meta=np.ndarray>
    FLDS            (time, ncol) float32 dask.array<chunksize=(4, 835586), meta=np.ndarray>
    ...              ...
    nsbase          (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
    nscur           (time) int32 dask.array<chunksize=(4,), meta=np.ndarray>
    nsteph          (time) int32 dask.array<chunksize=(4,), meta=np.ndarray>
    sol_tsi         (time) float64 dask.array<chunksize=(4,), meta=np.ndarray>
    time_bnds       (time, nbnd) object dask.array<chunksize=(4, 2), meta=np.ndarray>
    time_written    (time) |S8 dask.array<chunksize=(4,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              X2000-mp3a
    logname:           zarzycki
    host:              cheyenne4
    initial_file:      /glade/u/home/zarzycki/scratch/MPAS/3km_florida/x20.83...
    topography_file:   /glade/u/home/zarzycki/work/cesmfiles/topo/mpasa3-60-f...
    model_doi_url:     not_set
    time_period_freq:  hour_6
    history:           Tue Sep 20 16:03:23 2022: ncks -4 -L 1 -O MPAS.VR3.EXT...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

In [25]:
h3cn_ds.isel(time=23)

<xarray.Dataset>
Dimensions:       (ncol: 835586, ilev: 33, lev: 32, nbnd: 2)
Coordinates:
  * ilev          (ilev) float64 4.5e+04 4.291e+04 4.085e+04 ... 703.5 249.0 0.0
  * lev           (lev) float64 4.395e+04 4.188e+04 3.984e+04 ... 476.3 124.5
    time          object 2008-09-01 21:00:00
Dimensions without coordinates: ncol, nbnd
Data variables: (12/51)
    CLDTOT        (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    FLUT          (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    OMEGA500      (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    OMEGA850      (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    PRECC         (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    PRECL         (ncol) float32 dask.array<chunksize=(835586,), meta=np.ndarray>
    ...            ...
    nsbase        int32 0
    nscur         int32 dask.array<chunksize=(), meta=np.ndarray>
    nsteph        int32 dask.array<chunksize=(), meta=np.ndarray>
    sol_tsi       float64 dask.array<chunksize=(), meta=np.ndarray>
    time_bnds     (nbnd) object dask.array<chunksize=(2,), meta=np.ndarray>
    time_written  |S8 dask.array<chunksize=(), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              X2000-mp3a
    logname:           zarzycki
    host:              cheyenne4
    initial_file:      /glade/u/home/zarzycki/scratch/MPAS/3km_florida/x20.83...
    topography_file:   /glade/u/home/zarzycki/work/cesmfiles/topo/mpasa3-60-f...
    model_doi_url:     not_set
    time_period_freq:  hour_3
    history:           Tue Sep 20 17:10:09 2022: ncks -4 -L 1 -O MPAS.VR3.EXT...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

In [41]:
h3cr_ds

<xarray.Dataset>
Dimensions:       (time: 41, lat: 1351, lon: 1851, ilev: 33, nbnd: 2, lev: 32)
Coordinates:
  * ilev          (ilev) float64 4.5e+04 4.291e+04 4.085e+04 ... 703.5 249.0 0.0
  * lat           (lat) float64 10.0 10.02 10.04 10.06 ... 36.96 36.98 37.0
  * lev           (lev) float64 4.395e+04 4.188e+04 3.984e+04 ... 476.3 124.5
  * lon           (lon) float64 268.0 268.0 268.0 268.1 ... 305.0 305.0 305.0
  * time          (time) object 2008-08-30 00:00:00 ... 2008-09-04 00:00:00
Dimensions without coordinates: nbnd
Data variables: (12/52)
    CLDTOT        (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    FLUT          (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    OMEGA500      (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    OMEGA850      (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    PRECC         (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    PRECL         (time, lat, lon) float32 dask.array<chunksize=(8, 1351, 1851), meta=np.ndarray>
    ...            ...
    nsbase        (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    nscur         (time) int32 dask.array<chunksize=(8,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(8,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(8,), meta=np.ndarray>
    time_bnds     (time, nbnd) object dask.array<chunksize=(8, 2), meta=np.ndarray>
    time_written  (time) |S8 dask.array<chunksize=(8,), meta=np.ndarray>
Attributes: (12/17)
    Conventions:       CF-1.0
    source:            CAM
    case:              X2000-mp3a
    logname:           zarzycki
    host:              cheyenne4
    initial_file:      /glade/u/home/zarzycki/scratch/MPAS/3km_florida/x20.83...
    ...                ...
    remap_hostname:    cheyenne5
    remap_version:     5.0.3
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....
    title:             Regridded version of MPAS.VR3.EXT.CAM5.4CLM5.0.cam.h3....
    map_file:          /glade/u/home/zarzycki/scratch/map_mpasa3-60-florida_t...
    input_file:        /glade/u/home/zarzycki/scratch/X2000-mp3a/run/MPAS.VR3...

In [5]:
c_mesh_ds

<xarray.Dataset>
Dimensions:                   (Time: 1, nCells: 835586, nVertLevels: 58,
                               nEdges: 2506752, nVertices: 1671168, TWO: 2,
                               maxEdges: 7, maxEdges2: 14, vertexDegree: 3,
                               R3: 3, nMonths: 12, FIFTEEN: 15, TWENTYONE: 21,
                               nVertLevelsP1: 59, nSoilLevels: 4)
Dimensions without coordinates: Time, nCells, nVertLevels, nEdges, nVertices,
                                TWO, maxEdges, maxEdges2, vertexDegree, R3,
                                nMonths, FIFTEEN, TWENTYONE, nVertLevelsP1,
                                nSoilLevels
Data variables: (12/128)
    qv                        (Time, nCells, nVertLevels) float32 dask.array<chunksize=(1, 1000, 58), meta=np.ndarray>
    qc                        (Time, nCells, nVertLevels) float32 dask.array<chunksize=(1, 1000, 58), meta=np.ndarray>
    qr                        (Time, nCells, nVertLevels) float32 dask.array<chunksize=(1, 1000, 58), meta=np.ndarray>
    initial_time              |S64 ...
    xtime                     (Time) |S64 dask.array<chunksize=(1,), meta=np.ndarray>
    latCell                   (nCells) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    ...                        ...
    h_oml_initial             (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    u10                       (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    v10                       (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    q2                        (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    rh2                       (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
    t2m                       (Time, nCells) float32 dask.array<chunksize=(1, 1000), meta=np.ndarray>
Attributes: (12/59)
    model_name:                       mpas
    core_name:                        init_atmosphere
    source:                           MPAS
    Conventions:                      MPAS
    git_version:                      v7.3-370-g25365456
    on_a_sphere:                      YES
    ...                               ...
    config_pio_stride:                1
    config_block_decomp_file_prefix:  x1.40962.graph.info.part.
    config_number_of_blocks:          0
    config_explicit_proc_decomp:      NO
    config_proc_decomp_file_prefix:   graph.info.part.
    file_id:                          xer2uv295n

In [40]:
c_mesh_ds.v_init.values

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [19]:
h3cn_ds

<xarray.Dataset>
Dimensions:       (time: 41, ncol: 835586, ilev: 33, lev: 32, nbnd: 2)
Coordinates:
  * ilev          (ilev) float64 4.5e+04 4.291e+04 4.085e+04 ... 703.5 249.0 0.0
  * lev           (lev) float64 4.395e+04 4.188e+04 3.984e+04 ... 476.3 124.5
  * time          (time) object 2008-08-30 00:00:00 ... 2008-09-04 00:00:00
Dimensions without coordinates: ncol, nbnd
Data variables: (12/51)
    CLDTOT        (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    FLUT          (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    OMEGA500      (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    OMEGA850      (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    PRECC         (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    PRECL         (time, ncol) float32 dask.array<chunksize=(8, 835586), meta=np.ndarray>
    ...            ...
    nsbase        (time) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0
    nscur         (time) int32 dask.array<chunksize=(8,), meta=np.ndarray>
    nsteph        (time) int32 dask.array<chunksize=(8,), meta=np.ndarray>
    sol_tsi       (time) float64 dask.array<chunksize=(8,), meta=np.ndarray>
    time_bnds     (time, nbnd) object dask.array<chunksize=(8, 2), meta=np.ndarray>
    time_written  (time) |S8 dask.array<chunksize=(8,), meta=np.ndarray>
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              X2000-mp3a
    logname:           zarzycki
    host:              cheyenne4
    initial_file:      /glade/u/home/zarzycki/scratch/MPAS/3km_florida/x20.83...
    topography_file:   /glade/u/home/zarzycki/work/cesmfiles/topo/mpasa3-60-f...
    model_doi_url:     not_set
    time_period_freq:  hour_3
    history:           Tue Sep 20 17:10:09 2022: ncks -4 -L 1 -O MPAS.VR3.EXT...
    NCO:               netCDF Operators version 5.0.3 (Homepage = http://nco....

In [6]:
# This funtion splits a global mesh along longitude
#
# Examine the X coordinates of each triangle in 'tris'. Return an array of 'tris' where only those triangles
# with legs whose length is less than 't' are returned.
#
def unzipMesh(x,tris,t):
    return tris[(np.abs((x[tris[:,0]])-(x[tris[:,1]])) < t) & (np.abs((x[tris[:,0]])-(x[tris[:,2]])) < t)]

# Compute the signed area of a triangle
#
def triArea(x,y,tris):
    return ((x[tris[:,1]]-x[tris[:,0]]) * (y[tris[:,2]]-y[tris[:,0]])) - ((x[tris[:,2]]-x[tris[:,0]]) * (y[tris[:,1]]-y[tris[:,0]]))

# Reorder triangles as necessary so they all have counterclockwise winding order. CCW is what Datashader and MPL
# require.
#
def orderCCW(x,y,tris):
    tris[triArea(x,y,tris)<0.0,:] = tris[triArea(x,y,tris)<0.0,::-1]
    return(tris)

def createHVTriMesh(x,y,triangle_indices, var, var_name, n_workers=1):
    # Declare verts array
    # This is essentally an XYZ matrix (at the location of x and y, z=?) 3 X len(verts) array
    verts = np.column_stack([x, y, var])

    # Convert to pandas
    verts_df  = pd.DataFrame(verts,  columns=['Lon', 'Lat', var_name])
    tris_df   = pd.DataFrame(triangle_indices, columns=['v0', 'v1', 'v2'])

    # Convert to dask
    verts_ddf = dd.from_pandas(verts_df, npartitions=n_workers)
    tris_ddf = dd.from_pandas(tris_df, npartitions=n_workers)

    # Declare HoloViews element
    tri_nodes = hv.Nodes(verts_ddf, ['Lon', 'Lat', 'index'], [var_name])
    trimesh = hv.TriMesh((tris_ddf, tri_nodes))
    return(trimesh)

In [7]:
# Coordinates for hexagon centroids (triangle vertices)
lonCell = np.mod(np.rad2deg(c_mesh_ds['lonCell'].values) - 180.0, 360.0) - 180.0
latCell = np.rad2deg(c_mesh_ds['latCell'].values)
# Indices for triangle vertices (1671168 X 3)
tris = c_mesh_ds.cellsOnVertex.values - 1

# Coordinates for triangle centroids (hexagon vertices)
lonVertex = np.mod(np.rad2deg(c_mesh_ds['lonVertex'].values) - 180.0, 360.0) - 180.0
latVertex = np.rad2deg(c_mesh_ds['latVertex'].values)
# Indices for hexagon vertices (835586 X 7)
hexs = c_mesh_ds.verticesOnCell.values - 1

In [13]:
pairs = np.column_stack((latCell, lonCell))
pairs2 = np.column_stack((np.roll(latCell, -1), np.roll(lonCell, -1)))
dists = haversine_vector(pairs, pairs2, Unit.METERS)
pairs2

array([[ 26.286036, -88.7637  ],
       [ 28.567142, -74.573395],
       [  8.507519, -68.08179 ],
       ...,
       [ 27.992155, -78.98053 ],
       [ 28.010473, -79.00424 ],
       [ 33.414158, -80.24805 ]], dtype=float32)

In [16]:
dists

array([1140476.1   , 1422408.2   , 2331775.    , ...,    2949.685 ,
          3093.0051,  612497.9   ], dtype=float32)

In [10]:
test = np.meshgrid(lonCell[:10], latCell)

MemoryError: Unable to allocate 2.54 TiB for an array with shape (835586, 835586) and data type float32

In [9]:
np.meshgrid.m_as?

Object `np.meshgrid.m_as` not found.


In [ ]:
def make_take(indexer, slice_dim):
    return tuple(indexer if slice_dim % ndims == i else slice(None)  # noqa: S001
                 for i in range(ndims))
return take



In [6]:
# Calculated vorticity using dual (triangular) mesh
# This uses u and v values from vertices of triangles (centroids of hexagons) 
# and outputs vorticity at triangle centroids (hexagon vertices)
# Should get ~ 1671168 values
# According to MPAS documentation, vertical vorticity must be calculated on hex vertices

# u and v values at triangle vertices
wsp = h3cn_ds['U10'].values
u = np.multiply(np.abs(wsp), np.cos(latCell))*units('m/s')
v = np.multiply(np.abs(wsp), np.sin(latCell))*units('m/s')

pairs = np.column_stack((latCell, lonCell))
pairs2 = np.column_stack((np.roll(latCell, -1), np.roll(lonCell, -1)))
dists = haversine_vector(pairs, pairs2, Unit.METERS)

# delta_x = np.multiply(np.abs(dists), np.cos(latCell))*units.meter
# delta_y = np.multiply(np.abs(dists), np.sin(latCell))*units.meter

In [ ]:
# This submits dx, dy as a job to ROAR
import pickle
from dask_jobqueue import MoabCluster
from dask.distributed import Client

cluster = MoabCluster(
    walltime='08:00:00',
    cores=16,
    memory='64GB',
    job_extra_directives=['-A cmz5202_a_g_sc_default', '-m abe', '-M cnd5285@psu.edu', '-j oe', '-N dask_jup'])

client = Client(cluster)

start = time.time()
delta_x, delta_y = client.submit(mpcalc.lat_lon_grid_deltas(c_mesh_ds.lonCell, c_mesh_ds.latCell))

end = time.time()
print(f'Time elapsed: {end-start} seconds.')

with open('delta_x.pickle', 'wb') as out_x:
    pickle.dump(delta_x, out_x)
with open('delta_y.pickle', 'wb') as out_y:
    pickle.dump(delta_y, out_y)

In [12]:
# 1 CPU, 4 PPN (memory error ~850mb for 1gb per processor)
start = time.time()
delta_x, delta_y = mpcalc.lat_lon_grid_deltas(c_mesh_ds.lonCell[:10], c_mesh_ds.latCell[:10])
end = time.time()
print(f'Time elapsed: {end-start} seconds.')

Time elapsed: 0.028813600540161133 seconds.


In [17]:
delta_x.shape

(10, 9)

In [ ]:
# Taken from https://unidata.github.io/python-gallery/examples/500hPa_Vorticity_Advection.html

u = h1cn_ds.U.isel(time=3)[:, :10]
v = h1cn_ds.U.isel(time=3)[:, :10]

rel_vorticity = mpcalc.vorticity(uwnd, vwnd, delta_x, delta_y, dim_order='yx')
f = c_mesh_ds.fVertex

abs_vorticity = rel_vorticity + f
abs_vorticity = ndimage.gaussian_filter(abs_vorticity, sigma=3, order=0) * units('1/s')

vort_adv = mpcalc.advection(abs_vorticity, [uwnd_500, vwnd_500], (dx, dy), dim_order='yx') * 1e9

In [16]:
u = h1cn_ds.U.isel(time=3)[:, :10]
v = h1cn_ds.U.isel(time=3)[:, :10]

rel_vorticity = mpcalc.vorticity(u, v, delta_x, delta_y, dim_order='yx')

TypeError: too many positional arguments

In [15]:
h1cn_ds.U.isel(time=3)[:, :10]

<xarray.DataArray 'U' (lev: 32, ncol: 10)>
dask.array<getitem, shape=(32, 10), dtype=float32, chunksize=(32, 10), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 4.395e+04 4.188e+04 3.984e+04 ... 997.8 476.3 124.5
    time     object 2008-09-02 00:00:00
Dimensions without coordinates: ncol
Attributes:
    mdims:         1
    units:         m/s
    long_name:     Zonal wind
    cell_methods:  time: mean

In [ ]:
# Calculated vorticity using primal (hexagonal) mesh
# This uses u and v values from vertices of hexagons (centroids of triangles)
# and outputs vorticity at hexagon centroids (triangle vertices)
# Still should not need Delaunay triangulation because of MPAS dual mesh
# Should get ~ 835586 values
# According to Helms and Hart 2013, should get better quality output with polygon circulation integral vs triangular CI.